# Oversampling and Undersampling

In this mini notebook we discuss how to address the problem of incredibly rare targets in classification problems. Read through on your own and work through the code as it arises.

## Extreme Splits

In a number of problems that are of interest, for example detecting disease or identifying instances of fraud, the target we're interested in does not occur frequently enough to allow for the construction of a good model.

Two techniques have been developed to address this issue, where the fundamental idea is to take the original data and create a <i>balanced</i> data set, train the algorithm on this data set, then readjust the probabilities output from the algorithm to reflect the original unbalanced data.

One technique does this by creating a larger dataset, the other by creating a smaller dataset.

Suppose we have a dataset with $n$ observations, $m$ features, for simplicity a binary target in which $j$ observations are $0$ and $n-j$ are $1$. Here we are assuming that $j >> n-j$

## Undersampling

In undersampling we create a balanced data set by cutting down on the number of $0$ observations until we match the $n-j$ instances of $1$. Note that this is approach is only recommended when we have such a large number of observations that it is not computationally feasible to perform oversampling. This is because you'll be throwing out most of your data.

### Random Undersampling

The earliest and easiest to implemement undersampling techniques is to just randomly (with or without replacement) sample from class $0$ until we've gotten $n-j$ observations.

### Additional Undersampling Techniques

See the wikipedia article on undersampling, <a href="https://en.wikipedia.org/wiki/Oversampling_and_undersampling_in_data_analysis">https://en.wikipedia.org/wiki/Oversampling_and_undersampling_in_data_analysis</a>, for additional techniques.

## Oversampling

In oversampling we create a balanced data set by simulating additional observations of class $1$. For large data sets this can be computationally expensive.

### Random Oversampling

Again the earliest and easiest way to implement oversampling is to randomly sample from the $1$ observations with replacement.

### The SMOTE Algorithm

The Synthetic Minority Over-sampling Technique (SMOTE) algorithm adapts $k$ nearest neighbors. To oversample, take a sample from the minority class, and consider its $k$ nearest neighbors (in the feature space). Take the vector between the point and one of those $k$ neighbors, you multiply the vector by a random number in $(0,1)$ then add that to the observation. This creates a new synthetic data point. Repeat until you have enough samples of the minority class.

### Additional Undersampling Techniques

See the wikipedia article on oversampling, <a href="https://en.wikipedia.org/wiki/Oversampling_and_undersampling_in_data_analysis">https://en.wikipedia.org/wiki/Oversampling_and_undersampling_in_data_analysis</a>, for additional techniques.


## Adjusting Your Probabilities

In either case if you oversample or undersample you need to adjust your probabilities after fitting your model if you want to know the true probability in the true data distribution.

We first discuss what can be done in undersampling.

### Bayes Theorem in Undersampling

Let $U$ be the undersampled set, and $s$ be a random variable where for each observation $i$ we have:
$$
s_i = \left\lbrace \begin{array}{l l}
    1 & \text{ if } (X_i,y_i) \in U \\
    0 & \text{ else}
\end{array}
\right.
$$

In undersampling we are assuming that $P(s=1|y,X) = P(s=1|y)$ meaning that our undersampling procedure doesn't depend upon $X$. We can take advantage of Baye's Theorem and the Law of Total Probability to adjust our proability, but first we introduce more notation.

Let $P(y=1|s=1,X)$ denote the probability that an observation is class $1$ given that it was in $U$ and the feature values.

Then we have:
$$
P(y=1|s=1,X) = \frac{P(y=1|X)}{P(y=1|X) + P(s=1|y=0)P(y=0|X)},
$$
here $P(s=1|y=0)$ depends on the undersampling procedure that you utilize.

Using the fact that $P(y=0|X) = 1-P(y=1|X)$ and algebra we can get that:
$$
P(y=1|X) = \frac{P(s=1|y=0)P(y=1|s=1,X)}{1 + P(s=1|y=0)P(y=1|s=1,X) - P(y=1|s=1,X)}
$$

You get $P(s=1|y=0)$ from your undersampling procedure and you estimate $P(y=1|s=1,X)$ with you algorithm trained on the $U$ data. Plugging those into this equation allows you to estimate $P(y=1|X)$ in the original data set.

### Oversampling Adjustment

This can be done by appealing to ratios.

We'll do this through an example. Suppose the original data split has a $90/10$ split, meaning $.9$ of the data are of class $0$ and $.1$ of the data are of class $1$. You then oversample so that the split is $50/50$. 

Now say the algorithm predicts the probability that the specific instance is class $1$ is $.8$. 

We can think of adjusting by setting up a ratio.
$$
\frac{\text{og data set}}{\text{oversampled data set}} = \frac{.1}{.5} = \frac{x}{.8},
$$
the fraction represents what the probability is worth in the original data set compared to the oversampled data set. So we know in this example the og data has $.1$ of its observations as class $1$ and the oversampled set has $.5$ of its observations as class $1$. The algorithm says that in the oversampled data I give this point a value of $.8$. You then need to solve for what $.8$ is worth in the og data (this is what $x$ represents). Rearranging gives $x=.16$, similarly you can find for the probability the sample is class $0$ is found with:
$$
\frac{.9}{.5} = \frac{x}{.2},
$$
which gives $x = .36$.

Note that $.16 + .36 \neq 1$. So we need to scale these adjustments, dividing them by $.16+.36$.

So the adjusted probability is:
$$
\frac{.16}{.16 + .36} \approx .3077
$$

## A Useful Python Package

This seems like a hassle to have to code if you have to do undersampling or oversampling.

There is a python package that may be of use to you, `imblearn` <a href="https://imbalanced-learn.readthedocs.io/en/stable/api.html">https://imbalanced-learn.readthedocs.io/en/stable/api.html</a>.

This package seemingly applies a number of methods to perform undersampling or oversampling. So I would assume that it also offers methods of readjusting your probabilities after fitting the algorithm. However, I leave it to you to verify since I've never worked with the package myself.

## References

Imbalanced data sets are a popular research area because they are common in real life. I consulted the following while writing up this notebook.

<a href="http://blog.data-miners.com/2009/09/adjusting-for-oversampling.html">http://blog.data-miners.com/2009/09/adjusting-for-oversampling.html</a>

<a href="http://www.oliviercaelen.be/doc/ECML_under_v4.pdf">http://www.oliviercaelen.be/doc/ECML_under_v4.pdf</a>

<a href="http://www.data-mining-blog.com/tips-and-tutorials/overrepresentation-oversampling/">http://www.data-mining-blog.com/tips-and-tutorials/overrepresentation-oversampling/</a>

There are other approaches for readjusting as well that I don't know much about but seem to be popular.

<a href="https://en.wikipedia.org/wiki/Platt_scaling">https://en.wikipedia.org/wiki/Platt_scaling</a>

<a href="http://cseweb.ucsd.edu/~elkan/calibrated.pdf">http://cseweb.ucsd.edu/~elkan/calibrated.pdf</a>